<br>

# Introdução


In [ ]:
#!pip3 install pylint

In [ ]:
import re

import folium
import geopandas as gpd
import pandas as pd
import seaborn as sns
from folium import plugins
from open_geodata import lyr

from sp_tjsp_divadmin.my_paths import output_path_gpkg, output_path_map

<br>

## Parametros


In [ ]:
SIMPLIFY = 0.005

<br>

## Read Data


In [ ]:
# Read Data
gpkg_file = output_path_gpkg / 'sp_tjsp_divadmin.gpkg'
gdf = gpd.read_file(gpkg_file)

# Results
gdf.info()
gdf.head(2)

<br>

## Colors


In [ ]:
def create_colors(input_geojson, col_categories):
    gdf = gpd.read_file(input_geojson)
    # print(gdf.columns)

    # Column with category
    # col_categories = 'comarca_nome'

    # Set palette
    palette_polygon = 'Paired'

    # Get list of unique values
    categories = set(gdf[col_categories])
    categories = list(categories)
    categories.sort()
    # print(categories)

    # See the palette chosed
    pal = sns.color_palette(palette_polygon, n_colors=len(categories))

    # Set dictionary
    color_polygon = dict(zip(categories, pal.as_hex()))
    # color_polygon['Agência Ambiental de Ribeirão Preto'] = '#e699ff'
    # color_polygon['Agência Ambiental de Barretos'] = '#4da6ff'
    # color_polygon['Agência Ambiental de São Carlos'] = '#ff5050'
    # color_polygon['Agência Ambiental de Osasco'] = '#00264d'
    return color_polygon

In [ ]:
# dict_colors = create_colors(output_path_gpkg / 'sp_tjsp_divadmin.gpkg', 'comarca_nome')
# dict_colors

<br>

## Layer


In [ ]:
list(gdf.columns)

In [ ]:
def lyr_tjsp_rajs():
    # Input
    gdf = gpd.read_file(output_path_gpkg / 'sp_tjsp_divadmin.gpkg')
    # gdf['agencia'].replace('None', 'Sem Definição', inplace=True)
    gdf = gdf.to_crs(epsg=4326)
    gdf['geometry'] = gdf.simplify(SIMPLIFY)

    # Set dictionary
    colors = create_colors(
        output_path_gpkg / 'sp_tjsp_divadmin.gpkg', 'raj_nome'
    )

    # Popup
    # gdf['PopUp'] = gdf.apply(popup_html, axis=1)
    # gdf['popup'] = gdf.apply(popup_html, axis=1)

    # Delete Columns
    gdf.drop(
        [
            #'municipio_nome',
            'municipio_id',
            'municipio_cod_tjsp',
            'municipio_nome_tjsp',
            'comarca_id',
            'comarca_munsede',
            'comarca_nome',
            'comarca_nome_tjsp',
            'cj_id',
            'cj_nome',
            'cj_sigla',
            'raj_id',
            #'raj_nome',
            'raj_sigla',
            'raj_regiao',
            'juiz_diretor',
            'email',
            #'geometry'
        ],
        axis=1,
        inplace=True,
        errors='ignore',
    )
    # print(gdf.columns)

    # Layer
    return folium.GeoJson(
        gdf,
        name='Regiões Administrativas Judiciárias',
        smooth_factor=1.0,
        zoom_on_click=False,
        embed=False,
        control=True,
        show=False,
        style_function=lambda x: {
            'fillColor': colors[x['properties']['raj_nome']],
            'color': colors[x['properties']['raj_nome']],
            'weight': 1,
            'fillOpacity': 0.3,
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['municipio_nome', 'raj_nome'],
            aliases=['Munícipio', 'RAJ'],
            sticky=True,
            opacity=0.9,
            direction='right',
        ),
        # popup=folium.GeoJsonPopup(
        #     ['popup'],
        #     parse_html=False,
        #     max_width='400',
        #     show=False,
        #     labels=False,
        #     sticky=True,
        # localize=True,
        # style=f"""
        # background-color: #F0EFEF;
        # """,
        # ),
    )

In [ ]:
def lyr_tjsp_cjs():
    # Input
    gdf = gpd.read_file(output_path_gpkg / 'sp_tjsp_divadmin.gpkg')
    # gdf['agencia'].replace('None', 'Sem Definição', inplace=True)
    gdf = gdf.to_crs(epsg=4326)
    gdf['geometry'] = gdf.simplify(SIMPLIFY)

    # Set dictionary
    colors = create_colors(
        output_path_gpkg / 'sp_tjsp_divadmin.gpkg', 'cj_nome'
    )

    # Popup
    # gdf['PopUp'] = gdf.apply(popup_html, axis=1)
    # gdf['popup'] = gdf.apply(popup_html, axis=1)

    # Delete Columns
    gdf.drop(
        [
            #'municipio_nome',
            'municipio_id',
            'municipio_cod_tjsp',
            'municipio_nome_tjsp',
            'comarca_id',
            'comarca_munsede',
            'comarca_nome',
            'comarca_nome_tjsp',
            'cj_id',
            #'cj_nome',
            'cj_sigla',
            'raj_id',
            'raj_nome',
            'raj_sigla',
            'raj_regiao',
            'juiz_diretor',
            'email',
            #'geometry'
        ],
        axis=1,
        inplace=True,
        errors='ignore',
    )
    # print(gdf.columns)

    # Layer
    return folium.GeoJson(
        gdf,
        name='Circunscrições Judiciárias',
        smooth_factor=1.0,
        zoom_on_click=False,
        embed=False,
        control=True,
        show=True,
        style_function=lambda x: {
            'fillColor': colors[x['properties']['cj_nome']],
            'color': colors[x['properties']['cj_nome']],
            'weight': 1,
            'fillOpacity': 0.3,
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['municipio_nome', 'cj_nome'],
            aliases=['Munícipio', 'CJ'],
            sticky=True,
            opacity=0.9,
            direction='right',
        ),
        # popup=folium.GeoJsonPopup(
        #     ['popup'],
        #     parse_html=False,
        #     max_width='400',
        #     show=False,
        #     labels=False,
        #     sticky=True,
        # localize=True,
        # style=f"""
        # background-color: #F0EFEF;
        # """,
        # ),
    )

In [ ]:
def lyr_tjsp_comarcas():
    # Input
    gdf = gpd.read_file(output_path_gpkg / 'sp_tjsp_divadmin.gpkg')
    # gdf['agencia'].replace('None', 'Sem Definição', inplace=True)
    gdf = gdf.to_crs(epsg=4326)
    gdf['geometry'] = gdf.simplify(SIMPLIFY)

    # Set dictionary
    colors = create_colors(
        output_path_gpkg / 'sp_tjsp_divadmin.gpkg', 'comarca_nome'
    )

    # Popup
    # gdf['PopUp'] = gdf.apply(popup_html, axis=1)
    # gdf['popup'] = gdf.apply(popup_html, axis=1)

    # Delete Columns
    gdf.drop(
        [
            #'municipio_nome',
            'municipio_id',
            'municipio_cod_tjsp',
            'municipio_nome_tjsp',
            'comarca_id',
            'comarca_munsede',
            #'comarca_nome',
            'comarca_nome_tjsp',
            'cj_id',
            'cj_nome',
            'cj_sigla',
            'raj_id',
            'raj_nome',
            'raj_sigla',
            'raj_regiao',
            'juiz_diretor',
            'email',
            #'geometry'
        ],
        axis=1,
        inplace=True,
        errors='ignore',
    )
    # print(gdf.columns)

    # Layer
    return folium.GeoJson(
        gdf,
        name='Comarcas',
        smooth_factor=1.0,
        zoom_on_click=False,
        embed=False,
        control=True,
        show=False,
        style_function=lambda x: {
            'fillColor': colors[x['properties']['comarca_nome']],
            'color': colors[x['properties']['comarca_nome']],
            'weight': 1,
            'fillOpacity': 0.3,
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['municipio_nome', 'comarca_nome'],
            aliases=['Munícipio', 'Comarca'],
            sticky=True,
            opacity=0.9,
            direction='right',
        ),
        # popup=folium.GeoJsonPopup(
        #     ['popup'],
        #     parse_html=False,
        #     max_width='400',
        #     show=False,
        #     labels=False,
        #     sticky=True,
        # localize=True,
        # style=f"""
        # background-color: #F0EFEF;
        # """,
        # ),
    )

<br>

## PopUp


In [ ]:
# Add Field
def popup_html_comarcas(row):
    tel = (
        str(row['telefone'])
        .replace('-', '')
        .replace(')', '')
        .replace('(', '+55')
        .replace(' ', '')
    )
    fax = (
        str(row['fax'])
        .replace('-', '')
        .replace(')', '')
        .replace('(', '+55')
        .replace(' ', '')
    )

    html = """
    <div>
    <p><b>{}</b> pertence à:
    <h4><b>{}</b></h4></p>
    
    <p><b>Sede:</b><br>
    {}{}{}<br>
    {}
    {}
    {}</p>
    <p><b>Contatos:</b><br>
    {}
    {}
    {}</p>
    <p>{}</p>
    </div>
    """.format(
        ''
        if pd.isnull(row['municipio_nome'])
        else '{}'.format(row['municipio_nome']),
        '' if pd.isnull(row['agencia']) else '{}'.format(row['agencia']),
        '' if pd.isnull(row['endereco']) else '{}'.format(row['endereco']),
        '' if pd.isnull(row['numero']) else ', {}'.format(row['numero']),
        ''
        if pd.isnull(row['complemento'])
        else ' - {}'.format(row['complemento']),
        ''
        if pd.isnull(row['bairro'])
        else 'Bairro: {}<br>'.format(row['bairro']),
        ''
        if pd.isnull(row['municipio_sede'])
        else 'Município: {}<br>'.format(row['municipio_sede']),
        '' if pd.isnull(row['cep']) else 'CEP: {}<br>'.format(row['cep']),
        ''
        if pd.isnull(row['telefone'])
        else 'Telefone: <a href="tel:{}">{}</a><br>'.format(
            tel, row['telefone']
        ),
        ''
        if pd.isnull(row['fax'])
        else 'Fax: <a href="tel:{}">{}</a><br>'.format(fax, row['fax']),
        ''
        if pd.isnull(row['email'])
        else 'E-mail: <a href="mailto:{}">{}</a>'.format(
            row['email'], row['email']
        ),
        ''
        if pd.isnull(row['email'])
        else '<a href="{}" target="_blank">Conferir Dados</a>'.format(
            row['url']
        ),
    )

    html = html.replace('\n', '')
    html = re.sub('\s\s+', ' ', html)  # Remove Espaços no meio
    html = html.strip()
    return html

<br>

## Folium


In [ ]:
def get_map(input_geojson):
    # Input
    gdf = gpd.read_file(input_geojson)
    gdf = gdf.to_crs(epsg=4326)
    sw = gdf.bounds[['miny', 'minx']].min().values.tolist()
    ne = gdf.bounds[['maxy', 'maxx']].max().values.tolist()
    bounds = [sw, ne]

    # Zoom
    min_zoom = 5
    max_zoom = 21
    padding = 1

    # Create Map
    m = folium.Map(
        min_zoom=min_zoom,
        max_zoom=max_zoom,
        max_bounds=True,
        min_lat=bounds[0][0] * ((100 + padding) / 100),
        min_lon=bounds[0][1] * ((100 + padding) / 100),
        max_lat=bounds[1][0] * ((100 - padding) / 100),
        max_lon=bounds[1][1] * ((100 - padding) / 100),
        # tiles=None,
        # zoom_delta=0.1,
        # zoom_start=10,
    )

    # Add Layers
    # m.add_child(lyr.base.google_streets(min_zoom, max_zoom))
    # m.add_child(lyr.base.google_terrain(min_zoom, max_zoom))
    # m.add_child(lyr.base.google_hybrid(min_zoom, max_zoom))
    # m.add_child(lyr.base.google_satellite(min_zoom, max_zoom))

    # Monitoramento
    m.add_child(lyr_tjsp_comarcas())
    m.add_child(lyr_tjsp_cjs())
    m.add_child(lyr_tjsp_rajs())

    # Plugins
    m.fit_bounds(bounds)
    plugins.Fullscreen(
        position='topleft',
        title='Clique para Maximizar',
        title_cancel='Mininizar',
    ).add_to(m)
    folium.LayerControl(
        position='topright',
        collapsed=False,
    ).add_to(m)
    return m

In [ ]:
# Create Maps
m = get_map(output_path_gpkg / 'sp_tjsp_divadmin.gpkg')
m

In [ ]:
# Save
m.save(output_path_map / 'sp_tjsp_divadmin_map.html')
m